In [ ]:
import argparse
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
class CropLayer(object):
	def __init__(self, params, blobs):
		# initialize our starting and ending (x, y)-coordinates of
		# the crop
		self.startX = 0
		self.startY = 0
		self.endX = 0
		self.endY = 0

	def getMemoryShapes(self, inputs):
		# the crop layer will receive two inputs -- we need to crop
		# the first input blob to match the shape of the second one,
		# keeping the batch size and number of channels
		(inputShape, targetShape) = (inputs[0], inputs[1])
		(batchSize, numChannels) = (inputShape[0], inputShape[1])
		(H, W) = (targetShape[2], targetShape[3])

		# compute the starting and ending crop coordinates
		self.startX = int((inputShape[3] - targetShape[3]) / 2)
		self.startY = int((inputShape[2] - targetShape[2]) / 2)
		self.endX = self.startX + W
		self.endY = self.startY + H

		# return the shape of the volume (we'll perform the actual
		# crop during the forward pass
		return [[batchSize, numChannels, H, W]]

	def forward(self, inputs):
		# use the derived (x, y)-coordinates to perform the crop
		return [inputs[0][:, :, self.startY:self.endY,
				self.startX:self.endX]]

In [ ]:
print("[INFO] loading edge detector...")
protoPath = os.path.sep.join(["hed_model","deploy.prototxt"])
modelPath = os.path.sep.join(["hed_model","hed_pretrained_bsds.caffemodel"])
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
cv2.dnn_registerLayer("Crop", CropLayer)

In [ ]:
os.chdir("C:\Python\CMP_facades")

In [ ]:
dirlist = os.listdir('C:\Python\CMP_facades/img')

In [ ]:
dirlist

In [ ]:
for e,i in enumerate(dirlist):
    
    image = cv2.imread(os.path.sep.join(["img",i]))
    (H, W) = image.shape[:2]

#     plt.imshow(image, interpolation = 'bicubic')
#     plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
#     plt.show()

    print("[INFO] performing Canny edge detection...")
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(blurred, 30, 150)
    

    blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(W, H),
        mean=(104.00698793, 116.66876762, 122.67891434),
        swapRB=False, crop=False)

    print("[INFO] performing holistically-nested edge detection...")
    net.setInput(blob)
    hed = net.forward()
    hed = cv2.resize(hed[0, 0], (W, H))
    hed = (255 * hed).astype("uint8")
    #hed = cv2.Canny(hed, 30, 150)
    hed = cv2.bitwise_not(hed)

    cv2.imwrite(f'hed{i}', hed)
    cv2.waitKey(0)